In [3]:
import pandas as pd

file_path = r"C:\Users\aryan\Downloads\DOWNLOAD-123PGkwNz0F5D2N1bOH-dO2T4_X6xWvMO7n8RNk-wRs_eq_\imatinib(2).csv"

df = pd.read_csv(file_path,
                 sep=';',
                 quotechar='"',    # correctly handle quoted fields
                 engine='python',
                 on_bad_lines='skip')

# Strip quotes from column names
df.columns = df.columns.str.strip('"')

# Also strip quotes from string-type columns (optional but neat)
for col in df.select_dtypes(include='object').columns:
    df[col] = df[col].str.strip('"')

print(df.shape)
print(df.columns)
print(df.head())

# Check missing values in each column
missing_values = df.isnull().sum()
print("Missing values per column:\n", missing_values)

# Summary statistics for numeric columns
print(df.describe())

# Filter dataset if you want (example: only IC50)
ic50_df = df[df['Standard Type'] == 'IC50']

# Check how many rows in filtered data
print(f"Number of IC50 records: {len(ic50_df)}")

#Data filtering for ic50 values
df_ic50 = df[df['Standard Type'] == 'IC50']
df_ic50 = df_ic50.dropna(subset=['Smiles', 'Standard Value'])
df_ic50['Standard Value'] = pd.to_numeric(df_ic50['Standard Value'], errors='coerce')
df_ic50 = df_ic50.dropna(subset=['Standard Value'])
print(df_ic50.shape)


(248, 48)
Index(['Molecule ChEMBL ID', 'Molecule Name', 'Molecule Max Phase',
       'Molecular Weight', '#RO5 Violations', 'AlogP', 'Compound Key',
       'Smiles', 'Standard Type', 'Standard Relation', 'Standard Value',
       'Standard Units', 'pChEMBL Value', 'Data Validity Comment', 'Comment',
       'Uo Units', 'Ligand Efficiency BEI', 'Ligand Efficiency LE',
       'Ligand Efficiency LLE', 'Ligand Efficiency SEI', 'Potential Duplicate',
       'Assay ChEMBL ID', 'Assay Description', 'Assay Type', 'BAO Format ID',
       'BAO Label', 'Assay Organism', 'Assay Tissue ChEMBL ID',
       'Assay Tissue Name', 'Assay Cell Type', 'Assay Subcellular Fraction',
       'Assay Parameters', 'Assay Variant Accession', 'Assay Variant Mutation',
       'Target ChEMBL ID', 'Target Name', 'Target Organism', 'Target Type',
       'Document ChEMBL ID', 'Source ID', 'Source Description',
       'Document Journal', 'Document Year', 'Cell ChEMBL ID', 'Properties',
       'Action Type', 'Standard Text 

In [6]:
#generating morgan fingerprints
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit import RDLogger

# Disable RDKit warnings to avoid flooding notebook outputs
RDLogger.DisableLog('rdApp.*')

def smiles_to_fp(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol:
        return AllChem.GetMorganFingerprintAsBitVect(mol, radius=2, nBits=1024)
    return None

df_ic50['fingerprint'] = df_ic50['Smiles'].apply(smiles_to_fp)
df_ic50 = df_ic50[df_ic50['fingerprint'].notnull()]
print(df_ic50.shape)

#extracting unique targets from ic50 dataset
df_ic50['Target ChEMBL ID'] = df_ic50['Target ChEMBL ID'].str.strip()
unique_targets = df_ic50['Target ChEMBL ID'].unique()
print(f"Unique targets: {len(unique_targets)}")

#Molecular Fingerprint Generation using Morgan Fingerprints

from rdkit import Chem
from rdkit.Chem import AllChem
import pandas as pd
import numpy as np
# Disable RDKit warnings to avoid flooding notebook outputs
RDLogger.DisableLog('rdApp.*')

# Convert SMILES to RDKit molecule objects
df_ic50['mol'] = df_ic50['Smiles'].apply(Chem.MolFromSmiles)

# Generate Morgan fingerprints (radius=2, 2048 bits)
def mol_to_fp(mol):
    if mol:
        return AllChem.GetMorganFingerprintAsBitVect(mol, radius=2, nBits=2048)
    else:
        return np.nan

df_ic50['fingerprint'] = df_ic50['mol'].apply(mol_to_fp)

# Convert fingerprints to numpy arrays for ML input
def fp_to_array(fp):
    arr = np.zeros((1,), dtype=int)
    if fp:
        AllChem.DataStructs.ConvertToNumpyArray(fp, arr)
        return arr
    else:
        return np.nan

df_ic50['fp_array'] = df_ic50['fingerprint'].apply(fp_to_array)
print(df_ic50[['Smiles', 'fp_array']].head())




(233, 49)
Unique targets: 32
                                              Smiles  \
0  Cc1ccc(NC(=O)c2ccc(CN3CCN(C)CC3)cc2)cc1Nc1nccc...   
1  Cc1ccc(NC(=O)c2ccc(CN3CCN(C)CC3)cc2)cc1Nc1nccc...   
2  Cc1ccc(NC(=O)c2ccc(CN3CCN(C)CC3)cc2)cc1Nc1nccc...   
3  Cc1ccc(NC(=O)c2ccc(CN3CCN(C)CC3)cc2)cc1Nc1nccc...   
4  Cc1ccc(NC(=O)c2ccc(CN3CCN(C)CC3)cc2)cc1Nc1nccc...   

                                            fp_array  
0  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...  
1  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...  
2  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...  
3  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...  
4  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...  


In [11]:
#Protein Feature Extraction - Amino Acid Composition
import requests

def get_uniprot_sequence(target_chembl_id):
    # You might need a mapping from ChEMBL ID to UniProt ID (or find from ChEMBL directly)
    # For now, let's assume you have the UniProt ID or use a placeholder
    uniprot_id = "P00519"  # Example UniProt ID for BCR-ABL (replace accordingly)

    url = f"https://rest.uniprot.org/uniprotkb/{uniprot_id}.fasta"
    response = requests.get(url)
    if response.status_code == 200:
        fasta = response.text
        # Remove header line and join sequence lines
        seq = "".join(fasta.split("\n")[1:])
        return seq
    else:
        return None

# Example usage for a single target:
sequence = get_uniprot_sequence("some_chembl_id")
print(sequence)
from collections import Counter
import numpy as np

def aa_composition(seq):
    # Define the 20 standard amino acids
    amino_acids = 'ACDEFGHIKLMNPQRSTVWY'
    seq_len = len(seq)
    counts = Counter(seq)
    # Calculate frequency for each amino acid
    freq = np.array([counts.get(aa, 0) / seq_len for aa in amino_acids])
    return freq

# Your protein sequence (example)
protein_seq = "MLEICLKLVGCKSKKGLSSSSSCYLEEALQRPVASDFEPQGLSEAARWNSKENLLAGPSENDPNLFVALYDFVASGDNTLSITKGEKLRVLGYNHNGEWCEAQTKNGQGWVPSNYITPVNSLEKHSWYHGPVSRNAAEYLLSSGINGSFLVRESESSPGQRSISLRYEGRVYHYRINTASDGKLYVSSESRFNTLAELVHHHSTVADGLITTLHYPAPKRNKPTVYGVSPNYDKWEMERTDITMKHKLGGGQYGEVYEGVWKKYSLTVAVKTLKEDTMEVEEFLKEAAVMKEIKHPNLVQLLGVCTREPPFYIITEFMTYGNLLDYLRECNRQEVNAVVLLYMATQISSAMEYLEKKNFIHRDLAARNCLVGENHLVKVADFGLSRLMTGDTYTAHAGAKFPIKWTAPESLAYNKFSIKSDVWAFGVLLWEIATYGMSPYPGIDLSQVYELLEKDYRMERPEGCPEKVYELMRACWQWNPSDRPSFAEIHQAFETMFQESSISDEVEKELGKQGVRGAVSTLLQAPELPTKTRTSRRAAEHRDTTDVPEMPHSKGQGESDPLDHEPAVSPLLPRKERGPPEGGLNEDERLLPKDKKTNLFSALIKKKKKTAPTPPKRSSSFREMDGQPERRGAGEEEGRDISNGALAFTPLDTADPAKSPKPSNGAGVPNGALRESGGSGFRSPHLWKKSSTLTSSRLATGEEEGGGSSSKRFLRSCSASCVPHGAKDTEWRSVTLPRDLQSTGRQFDSSTFGGHKSEKPALPRKRAGENRSDQVTRGTVTPPPRLVKKNEEAADEVFKDIMESSPGSSPPNLTPKPLRRQVTVAPASGLPHKEEAGKGSALGTPAAAEPVTPTSKAGSGAPGGTSKGPAEESRVRRHKHSSESPGRDKGKLSRLKPAPPPPPAASAGKAGGKPSQSPSQEAAGEAVLGAKTKATSLVDAVNSDAAKPSQPGEGLKKPVLPATPKPQSAKPSGTPISPAPVPSTLPSASSALAGDQPSSTAFIPLISTRVSLRKTRQPPERIASGAITKGVVLDSTEALCLAISRNSEQMASHSAVLEAGKNLYTFCVSYVDSIQQMRNKFAFREAINKLENNLRELQICPATAGSGPAATQDFSKLLSSVKEISDIVQR"

protein_features = aa_composition(protein_seq)
print(protein_features)
print(f"Feature vector length: {len(protein_features)}")
import requests
import pandas as pd

# Your dataframe df_ic50 must already be loaded

# Get unique target ChEMBL IDs
unique_targets = df_ic50['Target ChEMBL ID'].unique()

# Dictionary to store target sequences
target_sequences = {}

# Loop through each target ID and fetch sequence from ChEMBL API
for target_id in unique_targets:
    url = f'https://www.ebi.ac.uk/chembl/api/data/target/{target_id}.json'
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        # Extract sequence - check if available under target_components
        try:
            components = data['target_components']
            if components and 'accession' in components[0]:
                # Get UniProt accession
                uniprot_id = components[0]['accession']

                # Fetch sequence from UniProt REST API
                uniprot_url = f'https://rest.uniprot.org/uniprotkb/{uniprot_id}.fasta'
                seq_response = requests.get(uniprot_url)
                if seq_response.status_code == 200:
                    fasta = seq_response.text
                    # Parse FASTA format to get sequence lines (skip header)
                    seq_lines = fasta.split('\n')[1:]
                    sequence = ''.join(seq_lines).strip()
                    target_sequences[target_id] = sequence
                else:
                    target_sequences[target_id] = None
            else:
                target_sequences[target_id] = None
        except (KeyError, IndexError):
            target_sequences[target_id] = None
    else:
        target_sequences[target_id] = None

# Map sequences back to the dataframe
df_ic50['Protein Sequence'] = df_ic50['Target ChEMBL ID'].map(target_sequences)

# Check how many sequences are fetched
print(f"Sequences fetched for {df_ic50['Protein Sequence'].notnull().sum()} out of {len(df_ic50)} entries")



MLEICLKLVGCKSKKGLSSSSSCYLEEALQRPVASDFEPQGLSEAARWNSKENLLAGPSENDPNLFVALYDFVASGDNTLSITKGEKLRVLGYNHNGEWCEAQTKNGQGWVPSNYITPVNSLEKHSWYHGPVSRNAAEYLLSSGINGSFLVRESESSPGQRSISLRYEGRVYHYRINTASDGKLYVSSESRFNTLAELVHHHSTVADGLITTLHYPAPKRNKPTVYGVSPNYDKWEMERTDITMKHKLGGGQYGEVYEGVWKKYSLTVAVKTLKEDTMEVEEFLKEAAVMKEIKHPNLVQLLGVCTREPPFYIITEFMTYGNLLDYLRECNRQEVNAVVLLYMATQISSAMEYLEKKNFIHRDLAARNCLVGENHLVKVADFGLSRLMTGDTYTAHAGAKFPIKWTAPESLAYNKFSIKSDVWAFGVLLWEIATYGMSPYPGIDLSQVYELLEKDYRMERPEGCPEKVYELMRACWQWNPSDRPSFAEIHQAFETMFQESSISDEVEKELGKQGVRGAVSTLLQAPELPTKTRTSRRAAEHRDTTDVPEMPHSKGQGESDPLDHEPAVSPLLPRKERGPPEGGLNEDERLLPKDKKTNLFSALIKKKKKTAPTPPKRSSSFREMDGQPERRGAGEEEGRDISNGALAFTPLDTADPAKSPKPSNGAGVPNGALRESGGSGFRSPHLWKKSSTLTSSRLATGEEEGGGSSSKRFLRSCSASCVPHGAKDTEWRSVTLPRDLQSTGRQFDSSTFGGHKSEKPALPRKRAGENRSDQVTRGTVTPPPRLVKKNEEAADEVFKDIMESSPGSSPPNLTPKPLRRQVTVAPASGLPHKEEAGKGSALGTPAAAEPVTPTSKAGSGAPGGTSKGPAEESRVRRHKHSSESPGRDKGKLSRLKPAPPPPPAASAGKAGGKPSQSPSQEAAGEAVLGAKTKATSLVDAVNSDAAKPSQPGEGLKKPVLPATPKPQSAKPSGTPISPAPVPSTLPSASSALAGDQPSST

In [14]:
#Data Preparation Summary
import numpy as np

# List of standard amino acids
amino_acids = list("ACDEFGHIKLMNPQRSTVWY")

def aa_composition(seq):
    seq = seq.upper()
    length = len(seq)
    comp = []
    for aa in amino_acids:
        comp.append(seq.count(aa) / length if length > 0 else 0)
    return np.array(comp)

# Apply to protein sequences, create new column 'protein_feat'
df_ic50['protein_feat'] = df_ic50['Protein Sequence'].apply(lambda x: aa_composition(x) if isinstance(x, str) else np.nan)

# Drop rows with missing fingerprints or protein features
df_clean = df_ic50.dropna(subset=['fp_array', 'protein_feat'])

# Prepare X and y for ML
X_mol = np.stack(df_clean['fp_array'].values)
X_prot = np.stack(df_clean['protein_feat'].values)
X = np.hstack([X_mol, X_prot])

# Binary labels: active if IC50 < 1000 nM else inactive
y = (df_clean['Standard Value'].astype(float) < 1000).astype(int)

print(f"Final dataset size: {X.shape[0]} samples with {X.shape[1]} features each.")

# Modeling and Performance Summary
#random forest apllied withSMOTE
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.model_selection import train_test_split

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Apply SMOTE to training data only
sm = SMOTE(random_state=42)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train)

# Train Random Forest
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train_res, y_train_res)

# Predict & evaluate
y_pred = clf.predict(X_test)
y_proba = clf.predict_proba(X_test)[:,1]

print(classification_report(y_test, y_pred))
print("ROC-AUC Score:", roc_auc_score(y_test, y_proba))

#Model Comparison: Random Forest vs. Support Vector Machine
#svm

from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, roc_auc_score
from imblearn.over_sampling import SMOTE

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# SMOTE only on training data
sm = SMOTE(random_state=42)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train)

# Define parameter grid
param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': ['scale', 0.01, 0.1, 1],
    'kernel': ['rbf'],
    'class_weight': [None, 'balanced']
}

# Grid Search with cross-validation
svm = SVC(probability=True, random_state=42)
grid = GridSearchCV(svm, param_grid, cv=5, scoring='roc_auc', n_jobs=-1)
grid.fit(X_train_res, y_train_res)

# Best parameters
print("Best Parameters:", grid.best_params_)

# Evaluate best model
best_svm = grid.best_estimator_
y_pred = best_svm.predict(X_test)
y_proba = best_svm.predict_proba(X_test)[:, 1]

print(classification_report(y_test, y_pred))
print("ROC-AUC Score:", roc_auc_score(y_test, y_proba))


Final dataset size: 140 samples with 2068 features each.
              precision    recall  f1-score   support

           0       0.67      0.25      0.36         8
           1       0.76      0.95      0.84        20

    accuracy                           0.75        28
   macro avg       0.71      0.60      0.60        28
weighted avg       0.73      0.75      0.71        28

ROC-AUC Score: 0.5249999999999999
Best Parameters: {'C': 0.1, 'class_weight': None, 'gamma': 1, 'kernel': 'rbf'}
              precision    recall  f1-score   support

           0       0.31      1.00      0.47         8
           1       1.00      0.10      0.18        20

    accuracy                           0.36        28
   macro avg       0.65      0.55      0.33        28
weighted avg       0.80      0.36      0.26        28

ROC-AUC Score: 0.45
